In [1]:
import pyidi
import os
import dill
import pickle
from sdypy import EMA
import numpy as np
import matplotlib.pyplot as plt 
from pixel_setter2 import PixelSetter
from scipy.signal import find_peaks
from pixel_setter import play_video
from scipy.spatial.distance import cdist
import matplotlib.animation as animation
from mpl_toolkits.mplot3d import Axes3D
from pixel_setter import animate_mode_shape
import math


%matplotlib inline

c:\Users\thijsmas\AppData\Local\anaconda3\Lib\site-packages\paramiko\transport.py:219: CryptographyDeprecationWarning: Blowfish has been deprecated
  "class": algorithms.Blowfish,


### Import displacement data

In [2]:
# root_disp = 'C:/Users/thijs/Documents/GitHub/pyidi_data/displacements'
root_disp = 'C:/Users/thijsmas/Documents/GitHub/pyidi_data/'
file_name_basis = 'Full_web_ecc0_new_Floc4_v1'
file_name_displacemnet = file_name_basis + '_d'
extension = '.pkl'
path = os.path.join(root_disp, file_name_displacemnet + extension)
root_drive_sim = os.path.normpath('G:/.shortcut-targets-by-id/1k1B8zPb3T8H7y6x0irFZnzzmfQPHMRPx/Illimited Lab Projects/Spiders/Simulations')

# Open displacement data
with open(path, 'rb') as f:
    # load the pickle data
    data = pickle.load(f)

# Open corresponding video
file_name_video = file_name_basis + "_S01.cihx"
folder_paths = ['D:/HSC', 'F:/', 'C:/Users/thijs/Documents/HSC/', 'H:/My Drive/PHD/HSC']  #, 'H:/My Drive/PHD/HSC'
for folder_path in folder_paths:
    for root, dirs, files in os.walk(folder_path, topdown=False):
        if file_name_video in files:
            file_path = os.path.join(root, file_name_video)
            print(file_path)
            break
video = pyidi.pyIDI(file_path)
fs_camera = video.info['Record Rate(fps)']
t_camera = np.arange(video.info['Total Frame']) / fs_camera
video.info

H:/My Drive/PHD/HSC\ecc0\web0\Full_web_ecc0_new_Floc4_v1_S01\Full_web_ecc0_new_Floc4_v1_S01.cihx


c:\Users\thijsmas\AppData\Local\anaconda3\Lib\site-packages\pyMRAW.py:103: UserWarning: Clipped footage! (Total frame: 10000, Original total frame: 87371)
  warnings.warn('Clipped footage! (Total frame: {}, Original total frame: {})'.format(cih['Total Frame'], cih['Original Total Frame'] ))


{'Date': '2023/9/16',
 'Camera Type': 'FASTCAM SA-Z type 2100K-M-64GB',
 'Record Rate(fps)': 8000.0,
 'Shutter Speed(s)': 8104.0,
 'Total Frame': 10000,
 'Original Total Frame': 87371,
 'Image Width': 1024,
 'Image Height': 512,
 'File Format': 'Mraw',
 'EffectiveBit Depth': 12,
 'EffectiveBit Side': 'Higher',
 'Color Bit': 16,
 'Comment Text': ''}

### import corresponding impact hammer data

In [3]:
root_impact = 'H:/My Drive/PHD/Data'
files = os.listdir(root_impact)

for file in files:
    if file_name_basis in file[16:-4]:
        path = os.path.join(root_impact, file)
        with open(path, 'rb') as f:
            impact_data = pickle.load(f)
        break
else:
    print(f"{file_name_basis} is not in files")

fs_force = impact_data['NI']['sample_rate']

In [4]:
%matplotlib qt
tp_lim = data['tracking points']
low_amplitude = np.max(np.linalg.norm(data['displacement'], axis=2),1)<25

td = data['displacement'] +  tp_lim.reshape(len(tp_lim),1,2)

if 'broken_ids_set' in locals():
    # broken ids: {930, 1220, 1417, 109, 977, 370, 469, 664, 1883, 1663}
    low_amplitude[nan_indices]  = False
    low_amplitude[list(broken_ids_set)]   = False
ani = play_video(video, range(1,9999), interval=30, points=td[low_amplitude, :, :])
# writer = animation.FFMpegWriter(fps=60)
# ani.save(os.path.join(root_disp,'anim',file_name_basis+'_anim.mp4'), writer=writer)


In [5]:
point = [265, 710 ]  # replace x and y with the coordinates of the point
# calculate the distances between the point and all points in tp_lim
distances = cdist([point], tp_lim)
# find the index of the nearest point
nearest_index = np.argmin(distances)
# get the coordinates of the nearest point
nearest_point = tp_lim[nearest_index]
play_video(video, range(1,5000), interval=30, points=td[nearest_index, :, :].reshape(1,-1,2))

In [13]:
%matplotlib qt
force_index = impact_data['NI']['channel_names'].index('Force')
accel_index = impact_data['NI']['channel_names'].index('Acceleration')
force = impact_data['NI']['data'][:,force_index]
accel = impact_data['NI']['data'][:,accel_index]

start_index = np.where(impact_data['NI']['time'] >= 1.5)[0][0]

impact_mean = np.mean(force[start_index:])
mean_accel = np.mean(accel[start_index:])

force -= impact_mean
accel = -(accel-mean_accel)/9.81

disp = data['displacement'][:, :, 0]
disp_nut = data['displacement'][nearest_index, :, 0]


### Shift the time signals such that the match approximately

In [14]:
# Add some zeros before the impact to reduce leakage. Due to the different sampling frequencies, the first common denominator is found and the number of zeros is calculated from there.
frames_per_gcd      = int(fs_camera / math.gcd(int(fs_camera), int(fs_force)))
n_zeros_before_cam  = frames_per_gcd
n_zeros_before_for  = int(n_zeros_before_cam * fs_force / fs_camera)

# Find the peaks in the displacement signal. 
disp_peaks, _       = find_peaks(disp_nut, height=0.5, distance=fs_camera*0.01)
threshold           = 0.001
near_zero_indices_cam   = np.where(disp_nut[:disp_peaks[0]] < threshold)[0]
first_cam_near_zero_index = near_zero_indices_cam[-1]

# Shift the time signal such that the impact - n_zeros_before_cam is at t=0, 
t_camera2           = np.round(t_camera - t_camera[first_cam_near_zero_index-n_zeros_before_cam], 7)
# Clip the time
time_cam_clipped    = t_camera2[first_cam_near_zero_index-n_zeros_before_cam:]
disp_nut_clipped    = disp_nut[first_cam_near_zero_index-n_zeros_before_cam:]
disp_clipped        = disp[:, first_cam_near_zero_index-n_zeros_before_cam:]
disp_clipped[:n_zeros_before_cam] = 0


# Prepare Force signal
force_peaks, _      = find_peaks(force, height=0.5)
threshold           = 0.001
near_zero_indices_for   = np.where(force[:force_peaks[0]] < threshold)[0]
first_for_near_zero_index = near_zero_indices_for[-1]
near_zero_indices_for_after   = (np.where(force[force_peaks[0]+1:force_peaks[0]+10] < threshold)[0]) + force_peaks[0]+1
first_for_near_zero_index_after = near_zero_indices_for[0]
t_force             = np.round(impact_data['NI']['time'] - impact_data['NI']['time'][first_for_near_zero_index - n_zeros_before_for],7)
t_force_index_end   = np.argmin(np.abs(t_force - t_camera2[-1]))

time_force_clipped    = t_force[first_for_near_zero_index- n_zeros_before_for:t_force_index_end]
force_clipped       = force[first_for_near_zero_index- n_zeros_before_for:t_force_index_end]

# Cut off signals such that they end at the same time. Loop is because sample ratios 
for i in range(1, frames_per_gcd+2): 
    if time_cam_clipped[-i] in time_force_clipped:
        index = np.where(time_cam_clipped[-i] == time_force_clipped)[0][0]
        time_cam_clipped = time_cam_clipped[:-i+1]
        disp_nut_clipped = disp_nut_clipped[:-i+1]
        disp_clipped = disp_clipped[:, :-i+1]
        time_force_clipped = time_force_clipped[:index+1]
        force_clipped = force_clipped[:index+1]
        break
time_cam_clipped[-1], time_force_clipped[-1]

(1.220625, 1.220625)

In [18]:
force_clipped_mod = np.copy(force_clipped)
force_clipped_mod[force_clipped_mod < 0.1] = 0

# force_mod[:first_for_near_zero_index+1] = np.zeros_like(force_mod[:first_for_near_zero_index+1])
# force_mod[first_for_near_zero_index_after:] = np.zeros_like(force_mod[first_for_near_zero_index_after:])
fig, ax = plt.subplots()

ax.plot(time_force_clipped, force_clipped,'r-')
ax.plot(t_force[force_peaks], force[force_peaks],'r*')

ax.plot(time_force_clipped, force_clipped_mod,'b-')
ax.plot(t_force[first_for_near_zero_index], force[first_for_near_zero_index],'go')
ax.plot(t_force[first_for_near_zero_index_after], force[first_for_near_zero_index_after],'ro')
ax.set_xlim([-0.003, 0.005])
plt.xlabel('Time (s)')
plt.ylabel('Force (N)')
plt.show()


In [23]:
tp_id = 50
fig, ax = plt.subplots(2,1,figsize=(20,5))
ax0_2 = ax[0].twinx()
ax1_2 = ax[1].twinx()
ax0_2.plot(t_force, force,'g-', label='Force')
# ax0_2.plot(t_force[force_peaks], force[force_peaks], 'gx')
# ax0_2.plot(t_force[first_for_near_zero_index], force[first_for_near_zero_index],'go')
# ax[0].plot(t_force, accel,'g--', label='Acceleration')
ax[0].plot(t_camera2, disp_nut,'r-', label='Displacement nut')
ax[0].plot(t_camera2, disp[tp_id,:],'r--', label=f'Displacement tp{tp_id}')
# ax[0].plot(t_camera2[disp_peaks], disp_nut[disp_peaks], 'rx')

ax[0].set_xlim([-0.02, t_camera[-1]])

ax[0].set_ylabel('pixels (-)')
ax0_2.set_ylabel('Force (N)')
ax[0].legend(loc='upper right')


ax1_2.plot(t_force, force,'g-', label='Force')
# ax1_2.plot(t_force[force_peaks], force[force_peaks], 'gx')
# ax1_2.plot(t_force[first_for_near_zero_index], force[first_for_near_zero_index],'go')
ax[1].plot(t_camera2, disp_nut,'r-', label='Displacement nut')
ax[1].plot(t_camera2, disp[tp_id,:],'r--', label=f'Displacement tp{tp_id}')
# ax[1].plot(t_camera2[disp_peaks], disp_nut[disp_peaks], 'rx')
ax[1].set_xlim([-0.002, 0.005])
ax[1].set_ylabel('pixels (-)')
ax1_2.set_ylabel('Force (N)')

# set legend to north east

ax[1].set_xlabel('Time (s)')
plt.show()
fig.savefig(os.path.join(root_drive_sim, 'signals', 'example impact'))


In [40]:
tp_id = 50
fig, ax = plt.subplots(2,1,figsize=(20,5))
ax0_2 = ax[0].twinx()
ax1_2 = ax[1].twinx()
ax0_2.plot(time_force_clipped, force_clipped,'g-', label='Force (raw))')
ax0_2.plot(time_force_clipped, force_clipped_mod,'b-', label='Force (zeros)')
ax[0].plot(time_cam_clipped, disp_nut_clipped,'r-', label='Displacement nut')
ax[0].plot(time_cam_clipped, disp_clipped[tp_id,:],'r--', label=f'Displacement tp{tp_id}')
ax[0].set_xlim([-0.02, time_cam_clipped[-1]])

ax[0].set_ylabel('pixels (-)')
ax0_2.set_ylabel('Force (N)')
ax[0].legend(loc='upper right')


ax1_2.plot(time_force_clipped, force_clipped,'g-', label='Force (raw)')
ax1_2.plot(time_force_clipped, force_clipped_mod,'b-', label='Force (zeros)')
ax[1].plot(time_cam_clipped, disp_nut_clipped,'r-', label='Displacement nut')
ax[1].plot(time_cam_clipped, disp_clipped[tp_id,:],'r--', label=f'Displacement tp{tp_id}')
# ax[1].plot(t_camera2[disp_peaks], disp_nut[disp_peaks], 'rx')
ax[1].set_xlim([-0.002, 0.005])
ax[1].set_ylabel('pixels (-)')
ax1_2.set_ylabel('Force (N)')

# set legend to north east

ax[1].set_xlabel('Time (s)')
plt.show()
fig.savefig(os.path.join(root_drive_sim, 'signals', 'example impact 2'))


n_d = len(disp_nut_clipped)
n_f = len(force_clipped)
n_d, n_f

extension_ratio = 5
n_d = int(np.ceil(n_d*extension_ratio))
n_f = int(np.ceil(n_f*extension_ratio))    

In [41]:
freq_camera = np.fft.rfftfreq(n_d, 1/fs_camera)
freq_force = np.fft.rfftfreq(n_f, 1/fs_force)
freq_camera[1], freq_force[1]

(0.16383370878558265, 0.16384524304777753)

In [42]:
Disp = np.fft.rfft(disp_clipped, n=n_d) / n_d
Force = np.fft.rfft(force_clipped_mod, n=n_f) / n_f

# Normalize the one-sided amplitude spectra:
Disp[ 1:] *= 2
Force[1:] *= 2
Disp.shape, Force.shape

((2155, 24416), (78123,))

In [43]:
n_freq = len(freq_camera)
freq_force = freq_force[:n_freq]

Disp = Disp[:n_freq]
Force = Force[:n_freq]

In [44]:
plt.figure(figsize=(15, 5))
plt.semilogy(freq_force, np.abs(Force), label='Force')
plt.semilogy(freq_camera, np.abs(Disp[tp_id]), label='Displacement')
plt.semilogy(freq_camera, np.abs(Disp[tp_id])/ np.abs(Force), label='output/input')
plt.semilogy(freq_camera, np.abs(Disp[nearest_index]), label='Displacement nut')
# plt.xlim(0, 200)
plt.xlabel('Frequency [Hz]')
plt.ylabel('Force [N], Displacment [m] amplitude spectra')
plt.legend()

In [45]:
T = time_force_clipped[-1] # acquisition period

S_xx = 1/T * np.conj(Disp) * Disp
S_ff = 1/T * np.conj(Force) * Force
S_xf = 1/T * np.conj(Disp) * Force
S_fx = 1/T * np.conj(Force) * Disp

H_1_cam = S_fx / S_ff
H_2_cam = S_xx / S_xf
H_v_cam = (H_1_cam * H_2_cam)**0.5


C:\Users\thijsmas\AppData\Local\Temp\ipykernel_19608\1994693530.py:9: RuntimeWarning: invalid value encountered in divide
  H_2_cam = S_xx / S_xf


In [47]:
plt.figure(figsize=(15, 5))
plt.semilogy(freq_force, np.abs(H_1_cam[tp_id]), label='H1')
plt.semilogy(freq_force, np.abs(H_2_cam[tp_id]), ':', label='H2')
# plt.semilogy(freq_camera, np.abs(Disp[tp_id])/ np.abs(Force), label='output/input')
# plt.semilogy(freq_camera, np.abs(Disp[tp_id])/ np.abs(Force) - np.abs(H_1_cam[tp_id]) , label='differences')
# plt.semilogy(freq_force, np.abs(H_v_cam[10]), '--', label='Hv')
# plt.semilogy(freq_force, np.abs(H_2_cam[10])-np.abs(H_1_cam[10]), label='diff')
plt.xlim(0, 100)
plt.xlabel('Frequency [Hz]')
plt.ylabel('H [pixel/N]')
plt.legend()
plt.savefig(os.path.join(root_disp,'FRFs',f'FRF_zero_padding_multiplier{extension_ratio}.png'))


In [ ]:
cam = EMA.Model(H_1_cam, freq_camera, lower=5, upper=200, pol_order_high=100)
# cam2 = EMA.Model(H_1_cam, freq_camera, lower=5, upper=100, pol_order_high=100)

In [ ]:
cam.get_poles(show_progress=True)
# cam2.get_poles(show_progress=True)

In [ ]:
cam.select_poles()
# cam.FRF_reconstruct
# cam2.select_poles()

In [ ]:
cam.print_modal_data()

In [ ]:
A = np.real(cam.A ) #,/ np.linalg.norm(cam.A, axis=0)
y_shift = (1024-768)/2
n_modes = len(cam.nat_freq) #1 #
broken_ids_set = set([])
for mode_number in range(n_modes):
    fig, ax = plt.subplots(subplot_kw={'projection': '3d'}, figsize=(15, 15))
    n_del = 30
    nan_indices = np.where(np.isnan(A[:,mode_number]))[0]
    not_nan_indices = np.where(~np.isnan(A[:,mode_number]))[0]

    lower_bound = np.percentile(np.abs(A[not_nan_indices,mode_number]), 5)
    upper_bound = np.percentile(np.abs(A[not_nan_indices,mode_number]), 95)
    values_within_bounds = A[(np.abs(A[:,mode_number]) >= lower_bound) & (np.abs(A[:,mode_number]) <= upper_bound), mode_number]
    mean_within_bounds = np.mean(values_within_bounds)

    A[:,mode_number] -= mean_within_bounds

    broken_ids = np.where(np.abs(A[:,mode_number]) >= 5)[0]
    broken_ids_set = broken_ids_set.union(set(broken_ids))
    multiplier = -1
    indices_to_plot = list(set(range(len(A))) - broken_ids_set)
    ax.plot(tp_lim[indices_to_plot,1], -tp_lim[indices_to_plot,0], np.zeros_like(tp_lim[indices_to_plot,0]), 'k.', markersize=1)
    ax.plot(tp_lim[indices_to_plot,1], -tp_lim[indices_to_plot,0], multiplier*A[indices_to_plot,mode_number], 'r.', markersize=1)
    # ax.plot(tp_lim[broken_ids,1], -tp_lim[broken_ids,0], multiplier*A[broken_ids,mode_number], 'b.', markersize=1)
    
    plt.xlim([0, 1024])
    plt.ylim([-768-y_shift, 0+y_shift])
    ax.set_zlabel('Normalized Mode Shape')
    ax.view_init(elev=30, azim=-90)
    plt.tight_layout()
    fig.savefig(os.path.join(root_disp,'mode_shapes',f'Mode_shape_init_{mode_number}_v1.png'))
# plt.show()

print(f'broken_ids_set: {broken_ids_set}')

In [ ]:
def animate_mode_shape(A, mode_number, tp_lim, multiplier=1, indices_to_plot = None, frames = range(200), interval = 30):
    if indices_to_plot is not None:
        A = A[indices_to_plot, :]
        tp_lim = tp_lim[indices_to_plot]
    A_imag = np.imag(A[:,mode_number])/np.linalg.norm(A[:,mode_number])
    A_real = np.real(A[:,mode_number])/np.linalg.norm(A[:,mode_number])
    fig, ax = plt.subplots(figsize=(15, 15), subplot_kw={'projection': '3d'})
    ax.set_title(f'Mode {mode_number+1} - {cam.nat_freq[mode_number]:.2f} Hz')
    ax.set_zlabel('Normalized mode shape')
    # Plot the inital position of the points
    ax.plot(tp_lim[:,1], -tp_lim[:,0], np.zeros_like(tp_lim[:,0]), 'k.', markersize=0.5)
    mode_plot, = ax.plot(tp_lim[:,1], -tp_lim[:,0], np.zeros_like(tp_lim[:,0]), 'r.', markersize=1)

    def update(frame):
        Z = multiplier*(A_imag * np.sin(frame * np.pi / 100) + A_real * np.cos(frame * np.pi / 100))
        mode_plot.set_data(tp_lim[:,1], -tp_lim[:,0])
        mode_plot.set_3d_properties(Z)
        return mode_plot

    ani = animation.FuncAnimation(fig, update, frames=frames, interval=interval)
    plt.show()
    return ani

for mode_number in [0]: # range(n_modes): #   
    indices_to_plot = list(set(range(len(A))) - broken_ids_set)
    ani = animate_mode_shape(cam.A, mode_number, tp_lim, 2, indices_to_plot = indices_to_plot)
    # writer = animation.FFMpegWriter(fps=60)
    # ani.save(os.path.join(root_drive_sim,'web mode shapes', f'Mode_shape_init_{mode_number}_v1.mp4'), writer=writer)
    # plt.close()
